<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

### How do I retrieve and download price feeds for historical backtesting?

In this notebook, we will retrieve all price feeds, back to 2011, for markets currently tradeable on the [Parcl Exchange](https://app.parcl.co/collection/active-markets)

**Note** This notebook will work with any of the dozens of daily price feeds available.

#### What will you create in this notebook?

In this notebook, we will create a spreadsheet for backtesting like the one below:

| date       | price_feed | parcl_id | name        | location_type | total_population | median_income |
|------------|------------|----------|-------------|---------------|------------------|---------------|
| 2024-06-17 | 415.29     | 5306725  | Denver City | CITY          | 710800           | 85853         |
| 2024-06-16 | 415.16     | 5306725  | Denver City | CITY          | 710800           | 85853         |
| 2024-06-15 | 414.92     | 5306725  | Denver City | CITY          | 710800           | 85853         |
| 2024-06-14 | 414.58     | 5306725  | Denver City | CITY          | 710800           | 85853         |
| 2024-06-13 | 414.61     | 5306725  | Denver City | CITY          | 710800           | 85853         |


#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/price_feed_trader_resources/download_sales_price_feed_data.ipynb)

### 1. Import the Parcl Labs Python Library

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs matplotlib

In [ ]:
import os
from parcllabs import ParclLabsClient

client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Search for Parcl Exchange Markets

In [ ]:
# lets get all US markets currently available to trade on the Parcl Exchange
# Now lets say you want all price feed markets that are on the parcl exchange
sales_pricefeed_markets = client.search.markets.retrieve(
    sort_by='PARCL_EXCHANGE_MARKET',
    limit=15,  # expand the default limit to 14, as of this writing, 14 markets are available
)

sales_pricefeed_markets

In [ ]:
# lets store the parcl_ids of the markets we are interested in
sales_pricefeed_parcl_ids = sales_pricefeed_markets['parcl_id'].tolist()

### 3. Retrieve Sales Price Feeds Data

In [ ]:
# lets retrieve data back to 2011 for these price feeds
START_DATE = '2011-01-01'

sales_price_feeds = client.price_feed.price_feed.retrieve(
    parcl_ids=sales_pricefeed_parcl_ids,
    start_date=START_DATE,
    limit=1000,  # expand the limit to 1000, these are daily series
    auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
)

sales_price_feeds.head()

In [ ]:
# 5384169 corresponds to Atlanta city
ATL = 5384169
sales_price_feeds.loc[sales_price_feeds['parcl_id'] == ATL].sort_values('date').plot(x='date', y='price_feed')

### 4. Create a Spreadsheet for Backtesting

In [ ]:
# lets pull in name, location_type, total_population, and median_income
df = sales_price_feeds.merge(sales_pricefeed_markets[['parcl_id', 'name', 'location_type', 'total_population', 'median_income']], on='parcl_id', how='inner')
df.head()

In [ ]:
# save to csv
df.to_csv('parcl_data.csv', index=False)